<a href="https://colab.research.google.com/github/Thiashopy04/PROSES-MINING/blob/main/BAGIAN_E_%E2%80%94_Conformance_Checking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pm4py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 20.4 MB/s eta 0:00:00


In [2]:
import pandas as pd

from pm4py.objects.log.util import dataframe_utils
from pm4py.objects.conversion.log import converter as log_converter

from pm4py.algo.discovery.inductive import algorithm as inductive_miner
from pm4py.algo.conformance.tokenreplay import algorithm as token_replay

In [3]:
df = pd.read_csv('/content/1767518350948_event_log_rumah_sakit_sintetis.csv')
df.head()

,case_id,activity,timestamp,resource,lifecycle,department,insurance_type,severity,cost_idr,variant
0,RS-00001,Pendaftaran,2025-12-11 19:17:08,Admin-01,complete,IGD,BPJS,Ringan,12290,rawat_jalan_sederhana
1,RS-00001,Triage,2025-12-11 19:53:08,Perawat-01,complete,IGD,BPJS,Ringan,0,rawat_jalan_sederhana
2,RS-00001,Pemeriksaan Dokter,2025-12-11 20:14:08,Dokter-Umum-01,complete,IGD,BPJS,Ringan,38944,rawat_jalan_sederhana
3,RS-00001,Resep Obat,2025-12-11 20:32:08,Dokter-Umum-02,complete,IGD,BPJS,Ringan,0,rawat_jalan_sederhana
4,RS-00001,Pembayaran,2025-12-11 21:15:08,Kasir-03,complete,IGD,BPJS,Ringan,0,rawat_jalan_sederhana


In [4]:
# konversi timestamp
df['timestamp'] = pd.to_datetime(df['timestamp'])

# urutkan event
df = df.sort_values(['case_id', 'timestamp'])

In [5]:
df_pm = df.rename(columns={
    'case_id': 'case:concept:name',
    'activity': 'concept:name',
    'timestamp': 'time:timestamp',
    'resource': 'org:resource'
})

df_pm = dataframe_utils.convert_timestamp_columns_in_df(df_pm)

In [6]:
event_log = log_converter.apply(df_pm)
type(event_log)

pm4py.objects.log.obj.EventLog

In [7]:
process_tree = inductive_miner.apply(event_log)

In [8]:
from pm4py.objects.conversion.process_tree import converter as pt_converter

net, im, fm = pt_converter.apply(process_tree)

In [9]:
replayed_traces = token_replay.apply(event_log, net, im, fm)

replaying log with TBR, completed traces ::   0%|          | 0/22 [00:00<?, ?it/s]

In [13]:
# hitung fitness tiap trace
trace_fitness = [trace['trace_fitness'] for trace in replayed_traces]

# fitness keseluruhan
average_fitness = sum(trace_fitness) / len(trace_fitness)

# persentase trace yang fit sempurna
perfect_fit_traces = sum(1 for f in trace_fitness if f == 1.0)
perc_fit_traces = perfect_fit_traces / len(trace_fitness)

print("Average Trace Fitness :", round(average_fitness, 4))
print("Percentage Fit Traces :", round(perc_fit_traces, 4))


Average Trace Fitness : 1.0
Percentage Fit Traces : 1.0


In [14]:
deviasi = []

for trace in replayed_traces:
    deviasi.append(trace['missing_tokens'] + trace['remaining_tokens'])

print("Rata-rata token deviasi:", sum(deviasi)/len(deviasi))

Rata-rata token deviasi: 0.0


In [15]:
trace_fitness = []

for i, trace in enumerate(replayed_traces):
    trace_fitness.append({
        'trace_index': i,
        'trace_fitness': trace['trace_fitness']
    })

df_fitness = pd.DataFrame(trace_fitness)

df_fitness.sort_values('trace_fitness').head(3)

,trace_index,trace_fitness
591,591,1.0
590,590,1.0
589,589,1.0


from matplotlib import pyplot as plt
_df_0['trace_index'].plot(kind='hist', bins=20, title='trace_index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['trace_fitness']
  ys = series['trace_index']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_1.sort_values('trace_fitness', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('trace_fitness')
_ = plt.ylabel('trace_index')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['trace_fitness']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'trace_fitness'}, axis=1)
              .sort_values('trace_fitness', ascending=True))
  xs = counted['trace_fitness']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_2.sort_values('trace_fitness', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('trace_fitness')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_3['trace_index'].plot(kind='line', figsize=(8, 4), title='trace_index')
plt.gca().spines[['top', 'right']].set_visible(False)

In [16]:
worst_traces = df_fitness.sort_values('trace_fitness').head(3)['trace_index']

for idx in worst_traces:
    print("Trace index:", idx)
    print(event_log[idx])
    print("-----")

Trace index: 591
{'attributes': {'concept:name': 'RS-00592'}, 'events': [{'concept:name': 'Pendaftaran', 'time:timestamp': Timestamp('2025-11-04 07:55:05+0000', tz='UTC'), 'org:resource': 'Admin-03', 'lifecycle': 'complete', 'department': 'Poli Bedah', 'insurance_type': 'BPJS', 'severity': 'Ringan', 'cost_idr': 8964, 'variant': 'rawat_jalan_lab'}, '..', {'concept:name': 'Selesai', 'time:timestamp': Timestamp('2025-11-04 12:07:05+0000', tz='UTC'), 'org:resource': 'System', 'lifecycle': 'complete', 'department': 'Poli Bedah', 'insurance_type': 'BPJS', 'severity': 'Ringan', 'cost_idr': 0, 'variant': 'rawat_jalan_lab'}]}
-----
Trace index: 590
{'attributes': {'concept:name': 'RS-00591'}, 'events': [{'concept:name': 'Pendaftaran', 'time:timestamp': Timestamp('2025-10-16 08:03:08+0000', tz='UTC'), 'org:resource': 'Admin-03', 'lifecycle': 'complete', 'department': 'Poli Penyakit Dalam', 'insurance_type': 'BPJS', 'severity': 'Ringan', 'cost_idr': 8572, 'variant': 'rawat_jalan_sederhana'}, '..'

✍️ Narasi Laporan

Conformance checking dilakukan menggunakan metode token-based replay.
Nilai fitness keseluruhan dihitung berdasarkan rata-rata fitness setiap trace.
Hasil analisis menunjukkan bahwa sebagian besar trace memiliki tingkat kesesuaian yang baik terhadap model proses, meskipun terdapat beberapa trace dengan fitness rendah yang mengindikasikan deviasi proses.